In [194]:
from datetime import datetime

import json
import os

import psycopg2
import pandas as pd

from tqdm import tqdm

In [126]:
INSTANCE_DIR = 'instance'

In [127]:
with open(os.path.join(INSTANCE_DIR, 'settings.json'), 'r') as f:
    settings = json.load(f)
settings.keys()

dict_keys(['host', 'user', 'database', 'password'])

In [128]:
DATA_DIR = os.path.join(INSTANCE_DIR, 'xlsx')

In [129]:
FILE_NAMES = {
    'providers':'s', 
    'details': 'p', 
    'supplies': 'sp',
}

In [130]:
files = {k:os.path.join(DATA_DIR, f'{v}.csv') for k, v in FILE_NAMES.items()}
files

{'providers': 'instance/xlsx/s.csv',
 'details': 'instance/xlsx/p.csv',
 'supplies': 'instance/xlsx/sp.csv'}

In [185]:
providers = pd.read_csv(files['providers'])
details = pd.read_csv(files['details'])
supplies = pd.read_csv(files['supplies'])

In [132]:
dbname = settings['database']
user = settings['user']
host = settings['host']
password = settings['password']

In [188]:
connection_string = f"dbname='{dbname}' user='{user}' host='{host}' password='{password}'"

In [189]:
connection = psycopg2.connect(connection_string)

In [190]:
cursor = connection.cursor()

In [136]:
def prepare_insert_providers_row(row):
    if pd.isna(row['Address']):
        address = 'NULL'
    else:
        address = f"'{row['Address']}'"
    
    return ("INSERT INTO providers "
            f"VALUES({row['SID']}, '{row['SName']}', '{row['SCity']}', {address}, {row['Risk']});")

In [137]:
def prepare_insert_details_row(row):
    if pd.isna(row['Color']):
        color = 'NULL'
    else:
        color = f"'{row['Color']}'"
    
    return ("INSERT INTO details "
            f"VALUES({row['PID']}, '{row['PName']}', '{row['PCity']}', {color}, {row['Weight']});")

In [181]:
def prepare_insert_supplies_row(row):
    date = datetime.strptime(row['ShipDate'], '%Y-%m-%d')
    return ("INSERT INTO supplies "
            f"VALUES({row['SPID']}, {row['SID']}, {row['PID']}, {row['Quantity']}, {row['Price']}, %s);"), (date,)


In [140]:
for command in tqdm(providers.apply(prepare_insert_providers_row, axis=1).values):
    try:
        cursor.execute(command)
    except:
        print(command)
        raise

In [138]:
for command in tqdm(details.apply(prepare_insert_details_row, axis=1).values):
    try:
        cursor.execute(command)
    except:
        print(command)
        raise

In [196]:
for command, args in tqdm(supplies.apply(prepare_insert_supplies_row, axis=1).values):
    try:
        cursor.execute(command, args)
    except:
        print(command, args)
        raise    

100%|██████████| 11803/11803 [00:00<00:00, 2096441.52it/s]


In [192]:
connection.commit()

In [193]:
cursor.close()
connection.close()